In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load data
test_data = pd.read_csv(r"MeDAL\pretrain_subset\test.csv")
train_data = pd.read_csv(r"MeDAL\pretrain_subset\train.csv")
valid_data = pd.read_csv(r"MeDAL\pretrain_subset\valid.csv")

# Save as Parquet (avoids pickling problem)
test_data.to_parquet("test.parquet", engine="pyarrow")
train_data.to_parquet("train.parquet", engine="pyarrow")
valid_data.to_parquet("valid.parquet", engine="pyarrow")

# Load directly into HuggingFace Dataset
from datasets import Dataset

test_dataset = Dataset.from_parquet("test.parquet")
train_dataset = Dataset.from_parquet("train.parquet")
valid_dataset = Dataset.from_parquet("valid.parquet")

# Combine into DatasetDict
from datasets import DatasetDict
medal_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "valid": valid_dataset
})


In [ ]:
medal_dataset["train"]

In [ ]:
# Save the dataset dictionary to disk in a folder called 'medal_dataset'
medal_dataset.save_to_disk("medal_dataset")
print("Dataset saved to 'medal_dataset' folder")

In [ ]:
# Save the dataset dictionary to disk in a folder called 'medal_dataset'
medal_dataset.save_to_disk("medal_dataset")
print("Dataset saved to 'medal_dataset' folder")

In [ ]:
from huggingface_hub import notebook_login

# # Create a smaller version of the dataset for easier upload
# small_medal_dataset = DatasetDict({
#     "train": train_dataset.select(range(10000)),
#     "test": test_dataset.select(range(1000)),
#     "valid": valid_dataset.select(range(1000))
# })

# Import required libraries

# Login to Huggingface Hub
notebook_login()

# Push to hub
medal_dataset.push_to_hub(
    "lutful2004/MeDAL-dataset",
    private=False,  # Set to True if you want a private dataset
    token=None  # Will use the token from notebook_login
)

print("Dataset uploaded successfully to Huggingface Hub")

In [ ]:
# Save the dataset dictionary to disk in a folder called 'medal_dataset'
medal_dataset.save_to_disk("medal_dataset")
print("Dataset saved to 'medal_dataset' folder")

In [ ]:
# import os

# # Create a directory to store smaller dataframes if it doesn't exist
# os.makedirs('smaller_datasets', exist_ok=True)

# # Take first 1000 rows from full_data
# full_data_small = full_data.head(1000)

# # Take first 10000 rows from train_data
# train_data_small = train_data.head(10000)

# # Take first 1000 rows from test_data and valid_data
# test_data_small = test_data.head(1000)
# valid_data_small = valid_data.head(1000)

# # Save the smaller dataframes to CSV files
# full_data_small.to_csv('smaller_datasets/full_data_small.csv', index=False)
# train_data_small.to_csv('smaller_datasets/train_data_small.csv', index=False)
# test_data_small.to_csv('smaller_datasets/test_data_small.csv', index=False)
# valid_data_small.to_csv('smaller_datasets/valid_data_small.csv', index=False)

# print("Smaller datasets saved to 'smaller_datasets' folder")

## Samall Sampling

In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict,load_from_disk,load_dataset


In [ ]:
dataset_path = "medal_dataset"
# dataset = load_from_disk(dataset_path)

dataset = load_dataset("lutful2004/MeDAL-dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 3000000
    })
    test: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 1000000
    })
    valid: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 1000000
    })
})

In [ ]:
from datasets import ClassLabel

# get unique labels
unique_labels = set()
for split in dataset.keys():
    # Convert to list first to make it hashable
    unique_labels.update(dataset[split]["LABEL"])
unique_labels = sorted(unique_labels)

# create ClassLabel feature
label_class = ClassLabel(num_classes=len(unique_labels), names=unique_labels)


In [ ]:
# cast the LABEL column
dataset = dataset.cast_column("LABEL", label_class)

# now you can do stratified sampling
sampled = DatasetDict({
    "train": dataset["train"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
    "test": dataset["test"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
    "valid": dataset["valid"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
})

print(sampled)


In [ ]:
from huggingface_hub import notebook_login

# # Create a smaller version of the dataset for easier upload
# small_medal_dataset = DatasetDict({
#     "train": train_dataset.select(range(10000)),
#     "test": test_dataset.select(range(1000)),
#     "valid": valid_dataset.select(range(1000))
# })

# Import required libraries

# Login to Huggingface Hub
notebook_login()

# Push to hub
sampled.push_to_hub(
    "lutful2004/MeDAL-dataset-small",
    private=False,  # Set to True if you want a private dataset
    token=None  # Will use the token from notebook_login
)

print("Dataset uploaded successfully to Huggingface Hub")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/300 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  16%|#6        | 33.5MB /  204MB            

...

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  25%|##4       | 33.5MB /  136MB            

README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Dataset uploaded successfully to Huggingface Hub
